In [ ]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow_datasets'

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Sample data
texts = ["This is a great movie.", "I didn't like this movie.", "The movie was okay."]
labels = [1, 2, 3]  # 1 for cat 1, 2 for cat 2 and 3 for cat 3

# 1. Tokenize the texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# 2. Pad sequences to ensure they have the same length
max_sequence_length = 10  # or choose based on your data
X = pad_sequences(sequences, maxlen=max_sequence_length)

# 3. One-hot encode the labels (3 categories)
y = to_categorical(labels, num_classes=4)  # num_classes should be 4 to include 0 indexing (1, 2, 3)

# 4. Define the RNN model with LSTM layer
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64, input_length=max_sequence_length),
    LSTM(64, activation='relu', return_sequences=True),  # LSTM layer
    GlobalAveragePooling1D(),  # Pool the LSTM output
    Dense(64, activation='relu'),
    Dense(4, activation='softmax')  # 4 categories: 0, 1, 2, 3
])

# 5. Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 6. Train the model
model.fit(X, y, epochs=10)

# Example of using the model to predict
test_text = ["I loved the film!"]
test_seq = tokenizer.texts_to_sequences(test_text)
test_seq_padded = pad_sequences(test_seq, maxlen=max_sequence_length)

pred = model.predict(test_seq_padded)
predicted_class = pred.argmax(axis=-1)  # Get the index of the highest probability
print(f"Predicted class: {predicted_class}")


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Sample data
texts = ["This is a great movie.", "I didn't like this movie.", "The movie was okay."]
labels = [1, 2, 3]  # 1 for cat 1, 2 for cat 2, and 3 for cat 3

# 1. Tokenize the texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# 2. Pad sequences to ensure they have the same length
max_sequence_length = 10  # You can adjust this based on the data
X = pad_sequences(sequences, maxlen=max_sequence_length)

# 3. One-hot encode the labels (3 categories)
y = to_categorical(labels, num_classes=4)  # num_classes should be 4 to include 0 indexing (1, 2, 3)

# 4. Load GloVe embeddings (assuming you've downloaded the file glove.6B.100d.txt)
def load_glove_embeddings(glove_file):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

# Load GloVe vectors (e.g., 100-dimensional vectors)
glove_file = 'glove.6B.100d.txt'  # Make sure to download and provide the correct path
embeddings_index = load_glove_embeddings(glove_file)

# 5. Prepare the embedding matrix
embedding_dim = 100  # Choose the dimension based on the GloVe file you use (e.g., 50, 100, 200)
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in tokenizer.word_index.items():
    if word in embeddings_index:
        embedding_matrix[i] = embeddings_index[word]

# 6. Define the model with GloVe embeddings
model = Sequential([
    # Use the GloVe embedding matrix (non-trainable by default)
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length,
              weights=[embedding_matrix], trainable=False),  # Set trainable=True if you want to fine-tune
    LSTM(64, activation='relu', return_sequences=True),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax')  # 4 categories: 0, 1, 2, 3
])

# 7. Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 8. Train the model
model.fit(X, y, epochs=10)

# Example of using the model to predict
test_text = ["I loved the film!"]
test_seq = tokenizer.texts_to_sequences(test_text)
test_seq_padded = pad_sequences(test_seq, maxlen=max_sequence_length)

pred = model.predict(test_seq_padded)
predicted_class = pred.argmax(axis=-1)  # Get the index of the highest probability
print(f"Predicted class: {predicted_class}")
